In [4]:

%matplotlib auto

# Import libraries:
from scipy.integrate import ode
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
import numpy as np
import time

Using matplotlib backend: TkAgg


In [5]:
### Simulation Model for Unicycle robot: Setup work here:

# Parameter setup:

# 1. Time range for simulation: 10 secs with 0.1 second time samples
t0 = 0 
tf = 20
delT = 0.05


# Time vector:
tVec = np.arange(t0, tf ,delT)


# 2. Initial state: 
x0 = np.array([ 4 , 5 , np.pi/2.1]).reshape((3,1))
#x0 = np.array([ 4 , 5 , 0]).reshape((3,1))



# 4. Input values - (Control Commands): 
 
# vel = 0.4*np.sin(2*np.pi*tVec)
vel = 3*np.ones((1,len(tVec)))
#omega = 0.05*(6*tVec - tVec**2)
omega = 0.8*np.ones((1,len(tVec)))



# 5. Process Noise simulation:
velStdDev = 0.0 # m/s
omegaStdDev = 0.0 # rad

epsVel =   velStdDev*np.random.randn(1,len(tVec))
epsOmega = omegaStdDev*np.random.randn(1,len(tVec))

# Estimated Input that goes to the Kalman Filter
uVec = np.vstack((vel,omega))

# The real input which actually goes into the simulated model:
# uVec = np.vstack((vel + epsVel , omega + epsOmega))

# True system dynamics function:
def x_dot(t, x , uVec, tVec): 
    
    x_dot = np.zeros((3,1))
    
    # Interpolate uVec: 
    v = np.interp(t, tVec, uVec[0,:])
    omega = np.interp(t, tVec, uVec[1,:])
    
    x_dot[0] = v*np.cos(x[2])
    x_dot[1] = v*np.sin(x[2])
    x_dot[2] = omega
    
    return x_dot



# Integrator for Ground Truth simulation:
r = ode(x_dot)

r.set_initial_value(x0, t0)

r.set_f_params(uVec,tVec)

# History of ODE integrator values and time: 
odeHist = []
odeHist.append(x0)

odeTime = []
odeTime.append(t0)

while r.successful() and r.t < tf:
    
    r.integrate(r.t+delT)
    
    odeHist.append(r.y.reshape((3,1)))
    odeTime.append(r.t)
    


xTrueHist = np.array(odeHist).reshape((-1,3))
tHist = np.array(odeTime)


interpXTrue = interp1d(tHist, xTrueHist , axis =0 , fill_value ="extrapolate")

# Get the reference output trajectory:
yRef = interpXTrue(tVec)
ud = uVec


plt.grid()
plt.xlabel(' X - (meters)')
plt.ylabel(' Y - (meters)')
plt.title( ' Ground Truth - Simulated Trajectory of Unicycle Model')



#plt.quiver(xTrueHist[0,::1],xTrueHist[1, ::1], uq, vq)
plt.plot(yRef[:,0], yRef[:,1], '*')



In [6]:
## 1. Utility functions to compute the solution to the Finite Horizon LQT problem 

# Sub-function: ricatti_derivative_func : 
# This returns the rate of change of the Pc matrix at any time instant: Pc includes the P matrix and the b vector


# Logic: 
# 1. Check if the parameters passed for A and B are functions or constants numpy arrays. If functions then this is a non-linear 
# system which needs to be linearized: 



# 1. Function giving partial derivative of dynamics w.r.t state vector 
def A_func(t,x,u): 
    
    """
    Inputs: 
    
    
    Outputs:
    
    
    """
    
    M , N = x.shape   
    
    # TO DO: Professional error handling:
    if N!=3: 
        
        print (" Dimensions of state vector incompatible with this function !!!")
        return 
    
    
    
    A_out = np.zeros((M,N,N))
    

    
    A_out[:,0,2] =  -np.sin(x[:,2])*u[0,:]
    A_out[:,1,2] =   np.cos(x[:,2])*u[0,:]
    
    return A_out

# 2. 
def B_func(t,x,u): 
    
    """
    
    
    """
    
    M , N = x.shape
    
    
    B_out = np.zeros((M,3,2))
    
    B_out[:,0,0] = np.cos(x[:,2])
    B_out[:,1,0] = np.sin(x[:,2])
    B_out[:,2,1] = np.ones((M))
    
    return B_out


# 3. 
def riccati_diff_eqn(t, Pc, A, B , Q, R_inv, C_p, yd): 
    
    """
    
    
    """
    
    
    # TO DO: Add additional check here : For linear vs non-linear system:
    # Extract interpolated values:
    
    A_int = A(-t)
    B_int = B(-t)
    yd_int = yd(-t)        
        
    
    N = A_int.shape[0]
    
    
    
    # Reshape the matrix Pc:     
    P = np.reshape(Pc[0:N*N] , (N,N))
    b = Pc[N*N:]
    
    
    # Calculate time rate of change of P matrix: 
    dPdt = np.dot(A_int.T, P) + np.dot(P,A_int) - np.linalg.multi_dot([P, B_int , R_inv , B_int.T, P]) + Q
    
    # Calculate time rate of change of b vector: 
    dbdt = np.dot( (A_int - np.linalg.multi_dot([B_int, R_inv, B_int.T, P])).T , b )  # + np.dot(Q, np.dot(C_p, yd_int))
    
    
    # Reshape back into a vector: 
    dPcdt = np.vstack( (dPdt.reshape((N*N,1)) , dbdt.reshape((N,1)) ))
    
    return dPcdt  
    
    



In [7]:
def finite_horizon_LQT(tVec,yd , A_func, B_func , Q, R_inv, MF,  C_p, ud): 
    
    """
    Inputs: 
    1. tVec  - Time vector of reference trajectory (M,)  - M is number of samples
    2. yd - Reference output trajectory to be tracked  (M,)
    3. A - System dynamics matrix (for linear system) or Partial derivative of system dynamics w.r.t state vector (for non-linear system)
    4. B - Input matrix (for linear system) or Partial derivative of system dynamics w.r.t input vector (for non-linear system)
    5. Q - Weighting matrix for states in Quadratic cost function
    6. R - Weighting matrix for inputs in Quadratic cost function
    7. C - Observation matrix - ()
    8. ud - Reference input trajectory
    
    Outputs: 
    1. Kp -
    2. Kb
    3. P - 
    4. b -   
    
    
    """
    
    # TO DO: Check whether A,B are numpy arrays or functions: 
    
    # Estimates reference state trajectory from reference output trajectory: 
    xd = yRef
    
    
    M , N = xd.shape
    I = R_inv.shape[0]
    
    # Use reference state trajectory to get linearized A(t) and B(t) matrices for the entire time vector: 
    A_ref = A_func(0,xd, ud)
    B_ref = B_func(0,xd, ud)
    
    
    # Create interploaters: 
    y_interp = interp1d(tVec, yRef, axis= 0,fill_value = "extrapolate")
    A_interp = interp1d(tVec, A_ref, axis=0 , fill_value = "extrapolate")
    B_interp = interp1d(tVec , B_ref , axis =0, fill_value = "extrapolate")
    
    # Setup variables for the Ricatti Differential Equation: 
    t0 = tVec[0]
    tf = tVec[-1]
    
    dt = tVec[1] - t0
    
    Pc_ini = np.zeros((N*N + N,1))
    
    Pc_ini[0:N*N]=MF.reshape((N*N,1))
    
    #print (Pc_ini)
    
    # History of ODE integrator values and time: 
    odeHist = []
    odeHist.append(Pc_ini)

    odeTime = []
    odeTime.append(-tf)
    
    # ODE solver for the Riccati differential equation: Backwards in time:
    odesolver = ode(riccati_diff_eqn)
    
    odesolver.set_initial_value(Pc_ini , -tf)
    
    odesolver.set_f_params(A_interp,B_interp,Q,R_inv, C_p, y_interp)
    
    
    jj =1
    
    while odesolver.successful() and odesolver.t < t0:
        
        odeHist.append(odesolver.integrate(odesolver.t+dt))
        odeTime.append(odesolver.t)       
    
        
    # Interpolate the ode history: 
    PcHist = np.array(odeHist).reshape((-1,N*N +N))
    tHist = np.array(odeTime)
    
    interpPc = interp1d(tHist, PcHist , axis =0, fill_value ="extrapolate")

    # Get the reference trajectory of Pc:
    Pc_int = interpPc(-tVec[::-1])
    
    # Recover P and b:
    P_rev = Pc_int[:, 0:N*N].reshape((-1, N,N))
    b_rev = Pc_int[:,N*N:].reshape((-1, N,1))
    
    
    # Flip P and b: 
    P = np.flip(P_rev, axis=0)
    b = np.flip(b_rev, axis=0)
    
#     print (M)
#     print (I)
#     print (N)
    
    Kp = np.zeros((M,I,N))
    Kb = np.zeros((M,I))
    
    # See if this step can be vectorized:
    for idx in range(M): 
        Kp[idx,:,:] = -np.linalg.multi_dot([R_inv , B_ref[idx, :, :].T , P[idx,:,:]]) 
        Kb[idx , :] = -np.linalg.multi_dot([R_inv, B_ref[idx, :, :].T , b[idx, :]]).reshape((I,))

    return Kp, Kb, P,b
    
    

In [10]:
##  Simulation cell for Testing the Implementation:  


# 0. Generate a disturbance about reference trajectory: 
nSamples = len(tVec)
stdDev = np.random.choice([0.1 ,0.2, 0.005,  5], nSamples , replace= True)


xDist = stdDev*np.random.randn(3,nSamples)


# Case 1: Apply reference input tajectory: Open Loop: 

# Open loop system dynamics: 
def x_dot_open(t, x , u, xDist): 
    
    x_dot = np.zeros((3,1))
    
    # Interpolate uVec: 
    u_int = u(t)
    xDist_int = xDist(t)
    
    
    v = u_int[0]
    omega = u_int[1]
    
    x_dot[0] = v*np.cos(x[2])
    x_dot[1] = v*np.sin(x[2])
    x_dot[2] = omega
    
    x_dot += xDist_int.reshape((3,1))
    
    return x_dot

xDist_interp = interp1d(tVec , xDist, axis =1, fill_value ="extrapolate")
uOpen_interp = interp1d(tVec, uVec, axis= 1 , fill_value = "extrapolate")

# Setup integrator:
openLoop = ode(x_dot_open)

openLoop.set_initial_value(x0, t0)

openLoop.set_f_params(uOpen_interp, xDist_interp)

count = 1

# History of ODE integrator values and time:
odeHist = []
odeHist.append(x0)

odeTime = []
odeTime.append(t0)

while openLoop.successful() and openLoop.t < tf:
    
    openLoop.integrate(openLoop.t+delT)
    
    odeHist.append(openLoop.y.reshape((3,1)))
    odeTime.append(openLoop.t)
    


xOpenHist = np.array(odeHist).reshape((-1,3))
tHist = np.array(odeTime)


interpXOpen = interp1d(tHist, xOpenHist , axis =0 , fill_value ="extrapolate")

# Get he reference output trajectory:
yOpen = interpXOpen(tVec)
    
    
# # Plot: 
# plt.grid()
# plt.xlabel(' X - (meters)')
# plt.ylabel(' Y - (meters)')
# plt.title( ' Simulated Trajectory of Unicycle Model with Disturbance- Open Loop system')


# #plt.quiver(xOpenHist[0,::1],xOpenHist[1, ::1], uq, vq)
# plt.plot(yOpen[:,0],yOpen[:,1], '*')


# # Compute the metric of performance: RMS Error from ref trajectory: 
RMS_Open = np.sqrt(np.sum ( (yOpen - yRef)**2, axis=0))
Term_error = np.sqrt( (yOpen[-1,:] - yRef[-1,:])**2)

print (" Open Loop Trajectory Error --> " , RMS_Open)
print (" Open Loop Terminal Error -->" , Term_error)


 Open Loop Trajectory Error -->  [  95.22791908  126.26028136   41.97074443]
 Open Loop Terminal Error --> [ 10.64096441   3.3031227    2.80466514]


In [11]:
# Case 2: Apply LQ feedback input : Closed Loop: 

# Define parameters: 
Q = np.diag([80,80,100])

MF = np.diag([8,8,4])

R = np.diag([1,1])

R_inv = np.linalg.inv(R)

C = np.identity(3)

C_p = np.linalg.pinv(C)

# Solve the finite Horizon LQR problem:
sTime = time.time()
Kp , Kb,  P , b = finite_horizon_LQT(tVec,yRef , A_func, B_func , Q, R_inv, MF, C_p, ud)
eTime = time.time()

print (" The time to compute the optimal control policy is: " , eTime - sTime , " secs ")


# Setup linear interpolator for feedback gain matrix, the ref input and ref state trajectory: 
Kp_interp = interp1d(tVec , Kp, axis=0 , fill_value = "extrapolate")
Kb_interp = interp1d(tVec , Kb, axis=0 , fill_value = "extrapolate")
uRef_interp = interp1d(tVec, uVec, axis= 1 , fill_value = "extrapolate")
xRef_interp = interp1d(tVec, yRef , axis = 0, fill_value ="extrapolate")



# Closed loop system dynamics: 
def x_dot_closed(t, x , uRef, xDist, Kp, Kb, xRef): 
    
    x_dot = np.zeros((3,1))
    
    # Interpolate: 
    uRef_int = uRef(t)
    xDist_int = xDist(t)
    xRef_int = xRef(t)
    Kp_int = Kp(t)
    Kb_int = Kb(t)
    
    
    # Calculate delta changes to ref input: 
    delX = x - xRef_int
    delU = np.dot(Kp_int, delX) + Kb_int
    
    
    u_LQT = uRef_int  + delU 
    
        
    v = u_LQT[0]
    omega = u_LQT[1]
    
    x_dot[0] = v*np.cos(x[2])
    x_dot[1] = v*np.sin(x[2])
    x_dot[2] = omega
    
    x_dot += xDist_int.reshape((3,1))
    
    return x_dot


# Setup integrator for closed loop:
closedLoop = ode(x_dot_closed)

closedLoop.set_initial_value(x0, t0)

closedLoop.set_f_params(uRef_interp, xDist_interp, Kp_interp, Kb_interp, xRef_interp)

# History of ODE integrator values and time:
odeHist = []
odeHist.append(x0)

odeTime = []
odeTime.append(t0)


sTime = time.time()
while closedLoop.successful() and closedLoop.t < tf:
    
    closedLoop.integrate(closedLoop.t+delT)
    
    odeHist.append(closedLoop.y.reshape((3,1)))
    odeTime.append(closedLoop.t)
    


xClosedHist = np.array(odeHist).reshape((-1,3))
tHist = np.array(odeTime)




interpXClosed = interp1d(tHist, xClosedHist , axis =0 , fill_value ="extrapolate")

# Get he reference output trajectory:
yClosed = interpXClosed(tVec)

eTime = time.time()

print (" The time to simulate the feedback control system is: " , eTime - sTime , " secs ")
    
# Plot: 
plt.plot(yRef[:,0], yRef[:,1], 'k-')
plt.plot(yOpen[:,0],yOpen[:,1], 'r*')
plt.plot(yClosed[:,0],yClosed[:,1], 'gs')

plt.grid()
plt.xlabel(' X - (meters)')
plt.ylabel(' Y - (meters)')
plt.title( ' Simulated Trajectory of Unicycle Model with Disturbance- LQT performance evaluation')
plt.legend([' Ground Truth Reference Trajectory' , 'Open Loop Trajectory' , ' LQT Feedback Trajectory',])


# Compute the metric of performance: RMS Error from ref trajectory: 
RMS_Closed = np.sqrt(np.sum ( (yClosed - yRef)**2, axis=0))
Term_error_c = np.sqrt( (yClosed[-1,:] - yRef[-1,:])**2)



print (" -------------------------------------------")

print (" --- Error Comparisons ---")
print (" Open Loop Trajectory Error --> " , RMS_Open)
print (" Open Loop Terminal Error -->" , Term_error)

print (" ---------------------------------------- ")

print (" Closed Loop Trajectory Error --> " , RMS_Closed)
print (" Closed Loop Terminal Error -->" , Term_error_c)



 The time to compute the optimal control policy is:  3.4607863426208496  secs 
 The time to simulate the feedback control system is:  5.151675462722778  secs 
 -------------------------------------------
 --- Error Comparisons ---
 Open Loop Trajectory Error -->  [  95.22791908  126.26028136   41.97074443]
 Open Loop Terminal Error --> [ 10.64096441   3.3031227    2.80466514]
 ---------------------------------------- 
 Closed Loop Trajectory Error -->  [ 4.57408315  5.2288019   4.23764025]
 Closed Loop Terminal Error --> [ 0.56587533  0.24799706  0.263709  ]


In [ ]:
class LinModel:
    
    # Constructor: 
    def __init__(self):
        
        params = {}
        
    
    # 1. Function describing the dynamic model: 
    def x_dot_func(self,t, x , u): 
    
        """
        t - Time  (sec)
        x - State vector - (2,1) - [x1 , x2]
        u - Input vector - (1,1) - [u]        """

        
        k = 20
        m = 300
        c = 10
        
        x_dot = np.zeros((3,1))
        
        A = np.array([[0 ,1], [-k/m -c/m]])
        
        B = np.array([0 , kt/m]).reshape((2,1))

        x_dot = np.dot(A,x) + np.dot(B,u)

        return x_dot
    
    # 2. Partial derivative of Dynamic model w.r.t state vector: 
    def F_func(self, t, x,u): 

        """
        t - Time vector (sec)
        x - State vector - (3,1) - [x , y, theta]
        u - Input vector - (2,1) - [v , omega]
        """

        F = np.zeros((3,3))

        F[0,2] = -np.sin(x[2])*u[0]
        F[1,2] = np.cos(x[2])*u[0]

        return F

    # 3. Partial derivative of Dynamic model w.r.t process noise: 
    def D_func(self,t,x,u): 

        """
        t - Time vector (sec)
        x - State vector - (3,1) - [x , y, theta]
        u - Input vector - (2,1) - [v , omega]
        """

        D = np.zeros((3,2))

        D[0,0] = np.cos(x[2])
        D[1,0] = np.sin(x[2])
        D[2,1] = 1

        return D